In [3]:
!pip install virtualenv

  Obtaining dependency information for virtualenv from https://files.pythonhosted.org/packages/14/19/e266f07cf55155d5f45170bbe08c486d8a9a9ae17bc8983acb1c019a8dd4/virtualenv-20.24.2-py3-none-any.whl.metadata
  Obtaining dependency information for distlib<1,>=0.3.7 from https://files.pythonhosted.org/packages/43/a0/9ba967fdbd55293bacfc1507f58e316f740a3b231fc00e3d86dc39bc185a/distlib-0.3.7-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 42.1 MB/s eta 0:00:00


In [4]:
!virtualenv venv

created virtual environment CPython3.10.12.final.0-64 in 1279ms
  creator CPython3Posix(dest=/content/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.2.1, setuptools==68.0.0, wheel==0.41.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [5]:
!source venv/bin/activate

In [6]:
!pip install --upgrade setuptools pip --user
!pip install --ignore-installed PyYAML
!pip install Pillow

!pip install nvidia-pyindex
!pip install --upgrade nvidia-tensorrt
!pip install pycuda

!pip install protobuf<4.21.3
!pip install onnxruntime-gpu
!pip install onnx>=1.9.0
!pip install onnx-simplifier>=0.3.6 --user
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu111/torch_stable.html

  Obtaining dependency information for PyYAML from https://files.pythonhosted.org/packages/29/61/bf33c6c85c55bc45a29eee3195848ff2d518d84735eb0e2d8cb42e0d285e/PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (705 kB)
/bin/bash: line 1: 4.21.3: No such file or directory


In [ ]:
import torch
torch.cuda.is_available()

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### **Convert pt file to onnx format**

In [8]:
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7

Cloning into 'yolov7'...
remote: Enumerating objects: 1191, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1191 (delta 2), reused 6 (delta 2), pack-reused 1185
Receiving objects: 100% (1191/1191), 74.22 MiB | 12.91 MiB/s, done.
Resolving deltas: 100% (513/513), done.
/content/yolov7


In [6]:
!python export.py --weights /content/gdrive/MyDrive/best_6.pt --grid  --img-size 640 640

Import onnx_graphsurgeon failure: No module named 'onnx_graphsurgeon'
Namespace(weights='/content/gdrive/MyDrive/best_6.pt', img_size=[640, 640], batch_size=1, dynamic=False, dynamic_batch=False, grid=True, end2end=False, max_wh=None, topk_all=100, iou_thres=0.45, conf_thres=0.25, device='cpu', simplify=False, include_nms=False, fp16=False, int8=False)
YOLOR 🚀 v0.1-126-g84932d7 torch 2.0.1+cu118 CPU

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
Model Summary: 314 layers, 36492560 parameters, 6194944 gradients
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

Starting TorchScript export with torch 2.0.1+cu118...
/content/yolov7/yolov7/yolov7/models/yolo.py:150: Trac

### **Converting Onnx format to tensorRT format**

In [9]:
# Download ONNX to TensorRT converter
%cd ../
!git clone https://github.com/Linaom1214/tensorrt-python.git

%cd tensorrt-python

/content
Cloning into 'tensorrt-python'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 323 (delta 57), reused 41 (delta 41), pack-reused 239
Receiving objects: 100% (323/323), 132.55 MiB | 16.61 MiB/s, done.
Resolving deltas: 100% (160/160), done.
/content/tensorrt-python


In [12]:
# Export TensorRT-engine model
!python export.py -o /content/drive/MyDrive/best_6_.onnx -e ./yolov7.trt -p fp16

Namespace(onnx='/content/drive/MyDrive/best_6_.onnx', engine='./yolov7.trt', precision='fp16', verbose=False, workspace=1, calib_input=None, calib_cache='./calibration.cache', calib_num_images=5000, calib_batch_size=8, end2end=False, conf_thres=0.4, iou_thres=0.5, max_det=100, v8=False)
[08/10/2023-08:11:13] [TRT] [I] [MemUsageChange] Init CUDA: CPU +340, GPU +0, now: CPU 364, GPU 173 (MiB)
[08/10/2023-08:11:21] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +732, GPU +172, now: CPU 1172, GPU 345 (MiB)
[08/10/2023-08:11:21] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage and speed up TensorRT initialization. See "Lazy Loading" section of CUDA documentation https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#lazy-loading
[08/10/2023-08:11:25] [TRT] [W] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32

### Moving tensorRt file from collab to google drive

In [13]:
#Moving from Collab to google drive
import shutil
colab_link = '/content/tensorrt-python/yolov7.trt'
gdrive_link = '/content/drive/MyDrive/For_RT'
shutil.copy(colab_link, gdrive_link)


'/content/drive/MyDrive/For_RT/yolov7.trt'